In [19]:
import openai
import os
import pandas as pd
import time
import json
import requests

/Users/Temp/Data4380/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
#importing module that wraps preprocessing steps
import LLM_Preprocessing as llmp

In [ ]:
#reimport if necessary (edit file after already imported)
import importlib
importlib.reload(llmp)

In [ ]:
#loading grouped dataset
df=llmp.llm_preprocessing('meta_Clothing_Shoes_and_Jewelry.jsonl.gz','Clothing_Shoes_and_Jewelry.jsonl.gz')

In [5]:
#grouping dataset if you have already created csv file and do not want to rerun preprocessing steps
df=pd.read_csv('amazon_watch_reviews.csv')
df=df.groupby("meta_title")["review_text"].apply(list).reset_index()

/var/folders/rs/56dc52116fdc6_zgn1vcs_fr0000gp/T/ipykernel_35204/3890872948.py:2: DtypeWarning: Columns (0,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('amazon_watch_reviews.csv')


In [16]:
client=openai.OpenAI(api_key='REDACTED INSERT YOUR OWN API KEY HERE')

#function to call GPT-4.5
def get_pros_cons(reviews):
    prompt=f"""Here are customer reviews for a watch:\n\n{reviews}\n\nPlease extract a list of PROS and CONS from the reviews.

Format:
Pros:
- ...
Cons:
- ..."""

    try:
        response=client.chat.completions.create(
            model="gpt-4-1106-preview", #only version of gpt-4 I could run successfully
            messages=[
                {"role": "system", "content": "You summarize reviews into pros and cons lists."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=400
        )
        return response.choices[0].message.content
    except Exception as e:
        print("Error:", e)
        return None

#sample (e.g. first 100)
sample_grouped=df.head(100).copy()

results=[]
for i, row in sample_grouped.iterrows():
    output=get_pros_cons(row['review_text'])
    results.append(output)
    time.sleep(1.1) 

sample_grouped["pros_cons"]=results

#save the results; jsonl file with formating that will work with OpenAI fine tuning
with open("labeled_watch_reviews.jsonl", "w") as f:
    for _, row in sample_grouped.iterrows():
        if row["pros_cons"]:
            json_record = {
                "messages": [
                    {"role": "system", "content": "You summarize customer reviews into pros and cons lists."},
                    {"role": "user", "content": row["meta_title"]},
                    {"role": "assistant", "content": row["pros_cons"]}
                ]
            }
            f.write(json.dumps(json_record) + "\n")


Error: Error code: 404 - {'error': {'message': 'The model `gpt-4-1106-preview` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}


In [26]:
sample_grouped.head()

,meta_title,review_text,pros_cons
0,!!! STACKER SALE !!! 50% OFF !!! LAST CHANCE T...,[I like this product a lot. I have about 5 tr...,Pros:\n- Attractive design\n- Sturdy construct...
1,"""A Promise Made"" Vintage Inspired Jewelry Book...",[I bought this for my best friend for her wedd...,"Pros:\n- Perfect for vintage-themed weddings, ..."
2,"""Courtney"" Mele & Co. Girl's Musical Ballerina...",[This is really cute. The drawer opens at bot...,Pros:\n- Attractive design with a cute feature...
3,"""DESPICABLE ME"" PURPLE BAND WATCH. Minion Wris...","[Cutest watch ever , bought for myself, little...",Pros:\n- Cute design\n- Liked by the purchaser...
4,"""Genevieve"" Wood Jewelry Box",[It looks fantastic and yet is still functiona...,Pros:\n- The watch has a fantastic look while ...


In [23]:
print(openai.__version__)

1.97.1


## Fine Tuning GPT-3.5
* Originally tried to do fine tuning in python environment; failed due to fine tuning not available for version of OpenAI API I am using (1.97.1)
* Switched to using the web option
    * Same concept, just using user friendly UI on website instead of assigning values through variables using python (batches, learning rate, epochs)

#### First Attempt
* Uses the prepared 100 samples that has labels (pros and cons list) generated by GPT-4.5
* Model = GPT 3.5 Turbo 1106
* Hyperparameters
    * Batches = 10
    * Learning Rate = 0.0001
    * Epochs = 10
* No validation file prepared for this test

Results
* Loss function looks like a heart rate ECG reading
* Accuracy values flucuated betweem 0.45 and 0.55
* Unsurprising that this model is doing so bad using a dataset of only 100
    * Also doing bad because lack of review text means model is just guessing at what the pros and cons are, will address this in next attempt